In [ ]:
# choose environment = crosscoders_p312

from utils import *
import torch
from trainer import Trainer

device = 'cuda:0'
local_dataset_path = "data/pile-lmsys-mix-1m-tokenized-gemma-2"
hf_dataset_path = "ckkissane/pile-lmsys-mix-1m-tokenized-gemma-2"
# torch.set_grad_enabled(False) # important for memory

base_model = HookedTransformer.from_pretrained(
    "gemma-2-2b", 
    device=device, 
    dtype=torch.bfloat16
)

chat_model = HookedTransformer.from_pretrained(
    "gemma-2-2b-it", 
    device=device,
    dtype=torch.bfloat16 
)

all_tokens = load_pile_lmsys_mixed_tokens(
    local_dataset_path=local_dataset_path,
    # hf_dataset_path=hf_dataset_path, # TODO: uncomment for first run
)

/home/ubuntu/miniconda3/envs/crosscoders_p312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  3.15it/s]


Loaded pretrained model gemma-2-2b into HookedTransformer


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.09it/s]


Loaded pretrained model gemma-2-2b-it into HookedTransformer
Loading data from disk


In [2]:
default_cfg = {
    "seed": 49,
    "batch_size": 4096, # 128
    "buffer_mult": 128,
    "lr": 5e-5,
    "num_tokens": 400_000_000,
    "l1_coeff": 2,
    "beta1": 0.9,
    "beta2": 0.999,
    "d_in": base_model.cfg.d_model,
    "dict_size": 2**14,
    "seq_len": 1024,
    "enc_dtype": "fp32",
    "model_name": "gemma-2-2b",
    "site": "resid_pre",
    "device": "cuda:0",
    "model_batch_size": 4, # 1
    "log_every": 100,
    "save_every": 30000,
    "dec_init_norm": 0.08,
    "hook_point": "blocks.14.hook_resid_pre",
    "wandb_project": "crosscoders-r1-distill-qwen",
    "wandb_entity": "varmaaastha6",
}
cfg = arg_parse_update_cfg(default_cfg)

trainer = Trainer(cfg, base_model, chat_model, all_tokens)

In IPython - skipped argparse


100%|██████████| 128/128 [01:06<00:00,  1.92it/s]
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: varmaaastha6 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
%%time
trainer.train()

 31%|███       | 29999/97656 [2:37:51<4:22:29,  4.30it/s] 

at i: saving model


 31%|███       | 30001/97656 [2:37:53<7:41:45,  2.44it/s]

Saved as version 0 in /home/ubuntu/crosscoders-r1-distill-qwen/checkpoints/version_1


 61%|██████▏   | 59999/97656 [5:15:42<1:55:40,  5.43it/s] 

...

at i: saving model


 92%|█████████▏| 90001/97656 [7:53:31<49:11,  2.59it/s]  

Saved as version 2 in /home/ubuntu/crosscoders-r1-distill-qwen/checkpoints/version_1


100%|██████████| 97656/97656 [8:33:49<00:00,  3.17it/s]


Saved as version 3 in /home/ubuntu/crosscoders-r1-distill-qwen/checkpoints/version_1
final: saved model
CPU times: user 8h 38min 51s, sys: 1min 17s, total: 8h 40min 9s
Wall time: 8h 33min 50s


In [1]:
import os
from pathlib import Path
import huggingface_hub

def push_to_hub(local_dir):
    if isinstance(local_dir, huggingface_hub.Repository):
        local_dir = local_dir.local_dir
    os.system(f"git -C {local_dir} add .")
    os.system(f"git -C {local_dir} commit -m 'Auto Commit'")
    os.system(f"git -C {local_dir} push origin main --force")


def upload_folder_to_hf(folder_path, repo_name=None, debug=False):
    """
    Uploads a folder to HuggingFace, and creates a repo for it.
    """
    folder_path = Path(folder_path)
    if repo_name is None:
        repo_name = folder_path.name
    repo_folder = folder_path.parent / (folder_path.name + "_repo")
    print(f"repo_folder: {repo_folder}")
    print(f"folder_path: {folder_path}")
    repo_url = huggingface_hub.create_repo(repo_name, exist_ok=True)
    repo = huggingface_hub.Repository(repo_folder, repo_url)

    for file in folder_path.iterdir():
        if debug:
            print(file.name)
        file.rename(repo_folder / file.name)
    push_to_hub(repo.local_dir)
   
model_name = "gemma-2-2b"
upload_folder_to_hf(folder_path=f"checkpoints/version_1", repo_name=f"crosscoders-{model_name}", debug=True)

repo_folder: checkpoints/version_1_repo
folder_path: checkpoints/version_1


/home/ubuntu/miniconda3/envs/crosscoders_p312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/crosscoders_p312/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/aastha6/crosscoders-gemma-2-2b into local empty directory.


2_cfg.json
2.pt
3_cfg.json
0.pt
3.pt
0_cfg.json
1.pt
1_cfg.json
[main 5d254c6] Auto Commit
 8 files changed, 16 insertions(+)
 create mode 100644 0.pt
 create mode 100644 0_cfg.json
 create mode 100644 1.pt
 create mode 100644 1_cfg.json
 create mode 100644 2.pt
 create mode 100644 2_cfg.json
 create mode 100644 3.pt
 create mode 100644 3_cfg.json


To https://huggingface.co/aastha6/crosscoders-gemma-2-2b
   5a9a0be..5d254c6  main -> main


In [ ]:
# details of successful training run

# Syncing run autumn-hill-21 to Weights & Biases (docs)
# View project at https://wandb.ai/varmaaastha6/crosscoders-r1-distill-qwen
# View run at https://wandb.ai/varmaaastha6/crosscoders-r1-distill-qwen/runs/ctd1eu0e

# Saved as version 3 in /home/ubuntu/crosscoders-r1-distill-qwen/checkpoints/version_1
# final: saved model
# CPU times: user 8h 38min 51s, sys: 1min 17s, total: 8h 40min 9s
# Wall time: 8h 33min 50s